<a href="https://colab.research.google.com/github/lakshmikanthreddybanapuram/QML-Lab/blob/main/qml_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TASK 3: Bell States and Entanglement Entropy**

### **Aim**
To construct **Bell States** using tensor products and to measure **entanglement entropy** in bipartite quantum systems.



### **1. Mathematical Model**

#### **1.1 Quantum Gates Representation**
**a. Hadamard Gate (H)**  
Transforms basis states as:  
$H|0\rangle = \frac{|0\rangle + |1\rangle}{\sqrt{2}}, \quad H|1\rangle = \frac{|0\rangle - |1\rangle}{\sqrt{2}}$

**b. Identity Gate (I)**  
Leaves the qubit state unchanged:  
$I|0\rangle = |0\rangle, \quad I|1\rangle = |1\rangle$

**c. Controlled-NOT (CNOT) Gate**  
Flips the target qubit if and only if the control qubit is in state $|1\rangle$:  
$|00\rangle \rightarrow |00\rangle, \quad |01\rangle \rightarrow |01\rangle, \quad |10\rangle \rightarrow |11\rangle, \quad |11\rangle \rightarrow |10\rangle$



#### **1.2 Bell States Construction**
Bell states are constructed by applying a Hadamard gate to the first qubit, followed by a CNOT operation between the two qubits.  
The four Bell states are:  
$|\Phi^{+}\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}, \quad |\Phi^{-}\rangle = \frac{|00\rangle - |11\rangle}{\sqrt{2}}, \quad |\Psi^{+}\rangle = \frac{|01\rangle + |10\rangle}{\sqrt{2}}, \quad |\Psi^{-}\rangle = \frac{|01\rangle - |10\rangle}{\sqrt{2}}$  
Each Bell state represents a **maximally entangled** two-qubit system.



#### **1.3 Partial Trace Operation**
Given a density matrix $\rho$ for a bipartite system $A \otimes B$, the **partial trace** over subsystem $B$ is defined as:  
$\rho_A = \mathrm{Tr}_B(\rho) = \sum_k \langle k|_B \rho |k\rangle_B$,  
where $\{|k\rangle_B\}$ is an orthonormal basis for subsystem $B$.


#### **1.4 Entanglement Entropy (Von Neumann Entropy)**
For a pure bipartite state $|\psi\rangle_{AB}$, the **entanglement entropy** is the von Neumann entropy of the reduced density matrix:  
$S(\rho_A) = -\sum_i \lambda_i \log_2(\lambda_i)$,  
where $\lambda_i$ are the eigenvalues of $\rho_A$.  
A **maximally entangled** state has $S = 1$, while a **product** (non-entangled) state has $S = 0$.



### **2. Algorithm**
1. Define quantum gates $H$, $I$, and $CNOT$.  
2. Construct the four Bell states using tensor products and gate operations.  
3. Convert the state vectors into density matrices $\rho = |\psi\rangle \langle\psi|$.  
4. Compute the **partial trace** over one subsystem to obtain $\rho_A$.  
5. Find eigenvalues of $\rho_A$ and calculate the **von Neumann entropy**.  
6. Verify that all Bell states yield entropy $S = 1$ and that the product state $|00\rangle$ has $S = 0$.





In [ ]:
import numpy as np
import cmath
print("\n" + "="*50)
print("TASK 4: COMMUTATION RELATIONS AND EULER ANGLES")
print("="*50)

import sympy as sp
I = sp.eye(2)
sx = sp.Matrix([[0, 1], [1, 0]])
sy = sp.Matrix([[0, -sp.I], [sp.I, 0]])
sz = sp.Matrix([[1, 0], [0, -1]])
paulis = {'X': sx, 'Y': sy, 'Z': sz}

def comm(A, B):
    return sp.simplify(A * B - B * A)

def anti(A, B):
    return sp.simplify(A * B + B * A)

print("\n=== Commutation relations ===")
for (a, b, k) in [('X', 'Y', 'Z'), ('Y', 'Z', 'X'), ('Z', 'X', 'Y')]:
    lhs = comm(paulis[a], paulis[b])
    rhs = 2 * sp.I * paulis[k]
    print(f"[{a},{b}] =\n{lhs}\nExpected:\n{rhs}\n")

print("\n=== Anti-commutation relations ===")
for i in ['X', 'Y', 'Z']:
    for j in ['X', 'Y', 'Z']:
        lhs = anti(paulis[i], paulis[j])
        rhs = 2 * (1 if i == j else 0) * I
        print(f"{{{i},{j}}} =\n{lhs} Expected:\n{rhs}\n")

# --- Part 2: Z–Y–Z Euler decomposition ---
def is_unitary(U, tol=1e-8):
    return np.allclose(U.conj().T @ U, np.eye(2), atol=tol)

def decompose_zyz(U, tol=1e-8):
    """Return (phi, alpha, beta, gamma) such that
    U = e^{i phi} Rz(alpha) Ry(beta) Rz(gamma)
    """
    U = np.array(U, dtype=complex)
    if not is_unitary(U):
        raise ValueError("Matrix is not unitary.")

    detU = np.linalg.det(U)
    phi = cmath.phase(detU) / 2
    U0 = U * np.exp(-1j * phi)

    detU0 = np.linalg.det(U0)
    U0 = U0 / np.sqrt(detU0)

    a = U0[0, 0]
    b = U0[0, 1]
    beta = 2 * np.arccos(min(1.0, max(0.0, abs(a))))

    if np.isclose(np.sin(beta / 2), 0, atol=tol):
        alpha = 2 * (-cmath.phase(a))
        gamma = 0.0
    else:
        phi1 = -cmath.phase(a)
        phi2 = -cmath.phase(-b)
        alpha = phi1 + phi2
        gamma = phi1 - phi2

    return float(phi), float(alpha), float(beta), float(gamma)

def Rz(theta):
    return np.array([[np.exp(-1j * theta / 2), 0],
                     [0, np.exp(1j * theta / 2)]], dtype=complex)

def Ry(theta):
    return np.array([[np.cos(theta / 2), -np.sin(theta / 2)],
                     [np.sin(theta / 2),  np.cos(theta / 2)]],
                     dtype=complex)

def reconstruct(phi, alpha, beta, gamma):
    return np.exp(1j * phi) @ (Rz(alpha) @ Ry(beta) @ Rz(gamma))

# --- Part 3: Test examples ---
def Rx(theta):
    return np.cos(theta / 2) * np.eye(2) - 1j * np.sin(theta / 2) * sx

examples = {
    "Rx(pi/3)": Rx(np.pi / 3),
    "Ry(pi/4)": Ry(np.pi / 4),
    "Rz(pi/2)": Rz(np.pi / 2),
    "H": (1 / np.sqrt(2)) * np.array([[1, 1], [1, -1]], dtype=complex),
    "S": np.array([[1, 0], [0, 1j]], dtype=complex),
    "T": np.array([[1, 0], [0, np.exp(1j * np.pi / 4)]], dtype=complex),
}

print("\n=== Z–Y–Z Euler Decomposition ===")
for name, U in examples.items():
    phi, alpha, beta, gamma = decompose_zyz(U)
    print(f"{name}:\n φ={phi:.6f}, α={alpha:.6f}, β={beta:.6f}, γ={gamma:.6f}\n")

# Optional: Use Cirq if available
try:
    import cirq
    print("\nCirq example decomposition for H gate:")
    # Create a qubit and turn H into an operation
    q = cirq.LineQubit(0)
    H_op = cirq.H(q)
    # Extract the unitary matrix of H
    U = cirq.unitary(H_op)
    # Perform Z–Y–Z decomposition
    phi, alpha, beta, gamma = decompose_zyz(U)
    print(f"Cirq H: φ={phi:.6f}, α={alpha:.6f}, β={beta:.6f}, γ={gamma:.6f}")
except ImportError:
    print("\nCirq not installed. Skipping Cirq examples.")



TASK 4: COMMUTATION RELATIONS AND EULER ANGLES

=== Commutation relations ===
[X,Y] =
Matrix([[2*I, 0], [0, -2*I]])
Expected:
Matrix([[2*I, 0], [0, -2*I]])

[Y,Z] =
Matrix([[0, 2*I], [2*I, 0]])
Expected:
Matrix([[0, 2*I], [2*I, 0]])

[Z,X] =
Matrix([[0, 2], [-2, 0]])
Expected:
Matrix([[0, 2], [-2, 0]])


=== Anti-commutation relations ===
{X,X} =
Matrix([[2, 0], [0, 2]]) Expected:
Matrix([[2, 0], [0, 2]])

{X,Y} =
Matrix([[0, 0], [0, 0]]) Expected:
Matrix([[0, 0], [0, 0]])

{X,Z} =
Matrix([[0, 0], [0, 0]]) Expected:
Matrix([[0, 0], [0, 0]])

{Y,X} =
Matrix([[0, 0], [0, 0]]) Expected:
Matrix([[0, 0], [0, 0]])

{Y,Y} =
Matrix([[2, 0], [0, 2]]) Expected:
Matrix([[2, 0], [0, 2]])

{Y,Z} =
Matrix([[0, 0], [0, 0]]) Expected:
Matrix([[0, 0], [0, 0]])

{Z,X} =
Matrix([[0, 0], [0, 0]]) Expected:
Matrix([[0, 0], [0, 0]])

{Z,Y} =
Matrix([[0, 0], [0, 0]]) Expected:
Matrix([[0, 0], [0, 0]])

{Z,Z} =
Matrix([[2, 0], [0, 2]]) Expected:
Matrix([[2, 0], [0, 2]])


=== Z–Y–Z Euler Decomposition ===
Rx

### **4. Result**
All four **Bell states** were successfully constructed using Hadamard and CNOT gates. The calculated **entanglement entropy** for each Bell state was found to be $S = 1$, confirming that they are **maximally entangled**. In contrast, the product state $|00\rangle$ exhibited zero entanglement entropy ($S = 0$), verifying that it is a **separable state**. These results confirm the theoretical predictions and validate the quantum mechanical behavior of entanglement.
